In [18]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

1.1.0
1.18.5


In [19]:
from modules.replace_dep import replace_with_dict, remove_college
replace_with_dict("언홍영")

'언론홍보영상학과'

In [20]:
remove_college("사회과학대학")

''

In [21]:
df_abstact_wrangle = pd.read_csv("data_wrangled/df_abstract_wrangle.csv", encoding="utf-8")
df_abstact_wrangle.sample(5)

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,REVIEW_LINK
3699,JP000012,간가쿠에서의 11개월,신문방송학과,학부,2008-2009,1,0.720381,http://oia.yonsei.ac.kr/partner/expReport.asp?...
2732,FR000009,INSA Lyon에서의 1년,생명공학과,학부,2018-2019,1,0.702207,http://oia.yonsei.ac.kr/partner/expReport.asp?...
10129,US000212,유매스에서의 한학기,아동가족학과,학부,2016,1,0.698352,http://oia.yonsei.ac.kr/partner/expReport.asp?...
903,CA000012,밴쿠버에서의 한 학기,생화학과,학부,2016,1,0.731432,http://oia.yonsei.ac.kr/partner/expReport.asp?...
10435,US000231,UPenn 경험 보고서,경영학과,학부,2011-2012,0,0.690176,http://oia.yonsei.ac.kr/partner/expReport.asp?...


In [22]:
departments = list(df_abstact_wrangle["학과"].unique())
print(len(departments), departments[:10])

1234 ['사회복지학과/언론홍보영상학부', '건축학과', '경영학과', '경제', '언론홍보영상학', '중어중문학과', '정치외교학과', '행정학과', '경제학부', '언론홍보영상학부']


In [23]:
df_abstact_wrangle["학과"] = df_abstact_wrangle["학과"].fillna(" ")
df_abstact_wrangle["학과"].isnull().values.any()

False

In [24]:
# 이중전공 두 개로 쪼개기
import re
import numpy as np
from itertools import chain

# return list from series of comma-separated strings
def split_double_majors(str):
    return re.split(',|/', str)

def chainer(series):
    # print(series)
    return list(chain.from_iterable(series.apply(split_double_majors)))

# calculate lengths of splits
# lens = df_abstact_wrangle['학과'].str.split('/').map(len)
lens = df_abstact_wrangle['학과'].apply(split_double_majors).map(len)
lens

0        2
1        1
2        1
3        1
4        1
        ..
11178    1
11179    2
11180    1
11181    1
11182    1
Name: 학과, Length: 11183, dtype: int64

In [25]:
# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'대학코드': np.repeat(df_abstact_wrangle['대학코드'], lens),
                    '제목': np.repeat(df_abstact_wrangle['제목'], lens),
                    '학과': chainer(df_abstact_wrangle['학과']),
                    '과정': np.repeat(df_abstact_wrangle['과정'], lens),
                    '년도': np.repeat(df_abstact_wrangle['년도'], lens),
                    'BERT_SCORE': np.repeat(df_abstact_wrangle['BERT_SCORE'], lens),
                    'RNN_SCORE': np.repeat(df_abstact_wrangle['RNN_SCORE'], lens),
                    'REVIEW_LINK': np.repeat(df_abstact_wrangle['REVIEW_LINK'], lens),})
res

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,REVIEW_LINK
0,730,리투아니아에서 행복했던 4개월,사회복지학과,학부,2019,1,0.752021,http://oia.yonsei.ac.kr/partner/expReport.asp?...
0,730,리투아니아에서 행복했던 4개월,언론홍보영상학부,학부,2019,1,0.752021,http://oia.yonsei.ac.kr/partner/expReport.asp?...
1,AR000004,아르헨티나 교환학생(어학연수프로그램),건축학과,학부,2013-1,0,0.399670,http://oia.yonsei.ac.kr/partner/expReport.asp?...
2,AT000001,쿠프슈타인에서 노후 50년 땡겨쓰기,경영학과,학부,2017-2018,0,0.416185,http://oia.yonsei.ac.kr/partner/expReport.asp?...
3,AT000001,평화로운 쿠프에서의 한 학기,경제,학부,2017ㅡ2018,1,0.816743,http://oia.yonsei.ac.kr/partner/expReport.asp?...
...,...,...,...,...,...,...,...,...
11179,US000301,Spring 2019 at Case,경영학과,학부,2019,1,0.512358,http://oia.yonsei.ac.kr/partner/expReport.asp?...
11179,US000301,Spring 2019 at Case,심리학과,학부,2019,1,0.512358,http://oia.yonsei.ac.kr/partner/expReport.asp?...
11180,US000301,"Case, 나의 대학생활 최고의 순간",창의기술경영학과,학부,2017-2018,1,0.699721,http://oia.yonsei.ac.kr/partner/expReport.asp?...
11181,US000303,미국 Liberal Arts College에서의 교환생활,언더우드학부 경제학과,학부,2020,0,0.686848,http://oia.yonsei.ac.kr/partner/expReport.asp?...


In [26]:
df_department = res[["대학코드", "학과", "REVIEW_LINK"]]
df_department.head()

,대학코드,학과,REVIEW_LINK
0,730,사회복지학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
0,730,언론홍보영상학부,http://oia.yonsei.ac.kr/partner/expReport.asp?...
1,AR000004,건축학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
2,AT000001,경영학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
3,AT000001,경제,http://oia.yonsei.ac.kr/partner/expReport.asp?...


In [27]:
# apply preprocessing module
df_department["학과"] = df_department["학과"].apply(lambda x: remove_college(x))
df_department["학과"] = df_department["학과"].apply(lambda x: replace_with_dict(x))
df_department.head()

,대학코드,학과,REVIEW_LINK
0,730,사회복지학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
0,730,언론홍보영상학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
1,AR000004,건축공학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
2,AT000001,경영학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...
3,AT000001,경제학과,http://oia.yonsei.ac.kr/partner/expReport.asp?...


In [28]:
from collections import Counter
print(Counter(df_department["학과"]))
# print(Counter(temp))

Counter({'경영학과': 2342, '영어영문학과': 960, '경제학과': 887, '정치외교학과': 610, '언론홍보영상학과': 564, '심리학과': 376, '행정학과': 245, '사회학과': 235, '중어중문학과': 209, '응용통계학과': 207, '전기전자공학과': 203, '교육학과': 200, '': 179, '국어국문학과': 171, '불어불문학과': 168, '사학과': 156, '의류환경학과': 152, '건축공학과': 137, '독어독문학과': 131, '노어노문학과': 120, '생활디자인학과': 114, '인문학부': 109, '기계공학과': 100, '생명공학과': 96, '사회복지학과': 90, '철학과': 89, '문화인류학과': 79, '법학과': 76, '정보산업공학과': 70, '문헌정보학과': 69, '신소재공학과\x1d': 69, '수학과': 61, '화공생명공학과': 60, 'UIC': 59, '신학과': 57, '식품영양학과': 55, '아동가족학과': 55, '화학과': 53, '컴퓨터과학과': 49, '도시공학과': 47, '의예과': 45, '주거환경학과': 45, '실내건축학과': 44, '의공학부': 44, '언더우드': 41, '스포츠레저학과': 40, '동아시아국제학부': 39, '생화학과': 38, '국제학과': 36, '토목환경공학과': 35, '화학공학과': 33, '물리학과': 30, '국제학대학원': 28, '주거환경': 24, '유럽어문학부': 24, '국제관계학과': 24, '기계전자공학부': 21, 'Underwood International College': 21, '천문우주학과': 20, '체육교육학과': 20, '시스템생물학과': 19, '문헌정보학': 18, '문화디자인경영': 18, '언더우드학부 경제학과': 18, '간호학과': 17, '대기과학과': 17, '산업공학과': 17, '생물학과': 16, '아시아학과': 16, '언더우드국제학부': 15, '지구시스템과

In [29]:
from math import nan

departments = df_department["학과"].unique()
# departments = temp.unique()

korean_deparments = []
english_departments = []
for department in departments:
    if type(department) == float:
        pass
    elif not re.search(r'[a-zA-Z]', department):
        korean_deparments.append(department)
    else:
        english_departments.append(department)


In [30]:
# print(len(korean_deparments), korean_deparments)
print(len(korean_deparments),sorted(korean_deparments))

409 ['', '-정치외교학과', '..', '1020011', '2012121230', '52-134', '간호학과', '건축공학과', '경영-경제 이중', '경영학과', '경영학과 이중', '경제학과', '계량위험관리', '계량위험관리학과', '과학기술정책', '교', '교육대학원', '교육대학원 영어교육학과', '교육학과', '교육학과 (영문학 이중)', '교육학과(영어영문 이중)', '교회음악과', '구세현', '국어국문학과', '국제', '국제개발협력', '국제경영', '국제관계', '국제관계학', '국제관계학과', '국제관계확과', '국제통상', '국제통상학', '국제통상학과', '국제학', '국제학 대학원', '국제학 대학원 국제경영', '국제학과', '국제학대학원', '국제학대학원 국제경영', '국제학대학원 국제통상', '국제학대학원 국제협력', '국제학대학원 지역학', '국제학부', '국제학부 경제학과', '국제학부 국제학', '국제학부 국제학과', '국제학부 비교문학과문화', '국제학부 비교문학과문화과', '국제학부 정치외교학', '국제학부 정치외교학과', '국제학부_정치외교학과', '국제협력', '글로벌 엘리트', '글로벌엘리트 학부', '글로벌엘리트학부', '글로벌인새대학', '글로벌인재대학', '글로벌인재학과', '글로벌인재학부', '글로벌인재학부 한국문화', '글로벌인재학부 한국문화·통상', '글로벌인재학부 한국문화과', '글로벌인재학부 한국문화통상 한국언어문화교육', '글로벌인제학부', '글로벌행정', '글로벌행정학과', '기계공학과', '기계전자공학부', '기계전자공학부 전기전자', '기악', '기악과', '나노과학공학', '나노과학공학과', '노어노문학과', '대기과학', '대기과학과', '대학원', '도시 환경 공학과', '도시공학', '도시공학과', '도시토목공학과', '독어독문학과', '동아시아 국제학부', '동아시아국제학과', '동아시아국제학부', '디자인', '디자인예술학부', '디지털예술학', '문리대학 생명과학과',

In [31]:
print(len(english_departments), english_departments)

178 ['EIC', 'Underwood International College', 'UD 국제학과', 'GSIS', 'EESE', 'Information and Interaction Design', 'Economics (UIC)', 'UIC - PSIR', 'UIC 아시아학과', 'UIC 문학과 문화학', 'UIC', 'TAD (IID & CDM)', 'International Studies', 'UIC경제학과', 'International Trade', 'Finance and Management', '언더우드학과 ud', 'Psychology', 'UIC(Quantitativive Risk Management)', 'UD ECON', '언더우드학부- 정치외교학과 (PSIR)', '국제학부 HASS SDC(지속개발협력)', '언더우드학부-국제학 (UD-IS)', 'UIC 생명과학공학', '국제학부 PSIR', 'Underwood Division', 'PSIR', '동아시아국제학부(EIC)', 'ASD', 'International studies', 'UIC 경제', 'Energy& Environmental Science Engineering', 'UIC ECON', '경영전문대학교 GMBA', 'Economics', '테크노아트학부 CDM', 'UIC국제학과', 'UIC 경제학과', 'UIC 비교문학', 'Social Sciences', 'philosophie', '에너지환경융합(EESE)', 'UIC 생명과학공학과', 'UIC 테크노아트학부', 'Quantitative Risk Management', 'Comparative Literature & Culture', 'UIC PSIR', 'UIC 문화디자인경영', '국제학과 (International Studies)', '계량위험관리(QRM)', 'Underwood International College - Economics', 'UIC - Economics', 'UIC IS', 'UIC Political S

In [32]:
univ_code = "JP000023"
one_univ_departments = df_department.loc[df_department["대학코드"]==univ_code]["학과"]
one_univ_departments

3885      언론홍보영상학과
3886        문헌정보학과
3887          경제학과
3888        영어영문학과
3889    정경대학 국제관계학
3890        역사문화학과
3891          교육학과
3892        국어국문학과
3893       생활디자인학과
3894      언론홍보영상학과
3895        사회체육학과
3896        영어영문학과
3897       기계전자공학부
3898      언론홍보영상학과
3899        보건행정학과
3900          행정학과
3901        영어영문학과
3902         도시공학과
3903        정치외교학과
3904              
3905       생활디자인학과
3906      언론홍보영상학과
Name: 학과, dtype: object

In [33]:
from collections import Counter, OrderedDict

dict_departments = Counter(one_univ_departments)
ordered_dict_departments= OrderedDict(dict_departments.most_common())
departments = list(ordered_dict_departments.keys())
number_of_students = list(ordered_dict_departments.values())
print(departments, number_of_students)

['언론홍보영상학과', '영어영문학과', '생활디자인학과', '문헌정보학과', '경제학과', '정경대학 국제관계학', '역사문화학과', '교육학과', '국어국문학과', '사회체육학과', '기계전자공학부', '보건행정학과', '행정학과', '도시공학과', '정치외교학과', ''] [4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [34]:
from collections import Counter, OrderedDict

univ_code = "DE000003"
one_univ_departments = df_department.loc[df_department["대학코드"]==univ_code]["학과"]
one_univ_departments

dict_departments = Counter(one_univ_departments)
ordered_dict_departments= OrderedDict(dict_departments.most_common())
departments = list(ordered_dict_departments.keys())
number_of_students = list(ordered_dict_departments.values())
print(departments, number_of_students)

['독어독문학과', '영어영문학과', '경영학과', '정치외교학과', '생활디자인학과', '사학과', '문화인류학과', 'philosophie', '사회복지학과', '교육학과', '심리학과', '인문학부', '유럽어문학부', ''] [32, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
